In [1]:
from manim import *
from manim_slides import Slide
from manim.utils.color import Colors
import json
from scipy.interpolate import interp1d

Manim Community v0.16.0.post0

In [ ]:
%%manim StonePath

with open('./ideas/positions_stone.json', 'r') as f:
    path = json.load(f)
    path = np.array([[-p[0],p[1],0.0] for p in path])*0.018+0.1
    f = interp1d(np.linspace(0,1,3001), path, axis=0)

class StonePath(Slide, MovingCameraScene):
    def construct(self):
        
        def update_bee(bee, dt):
            new_pos = f(tracker.get_value())
            tsub1 = f(np.maximum(0, tracker2.get_value()))
            
            diff = new_pos-tsub1
            angle = np.arctan2(diff[1], diff[0])
            bee.points = bee.original_points.copy()
                
            bee.rotate(angle-PI/2)
            bee.move_to(new_pos)
            
            tracker2.set_value(tracker.get_value())
            
        title = Title(r"Example path: Stone model").set_z_index(100).shift(UP*3+RIGHT*5)
        
        outbound = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        inbound = ParametricFunction(f, t_range=[0.5,1], color=ORANGE)
        
        outbound2 = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        
        label1 = Text("Outbound").move_to([10,5,0])
        line1 = Line([0,0,0],[1,0,0],color=BLUE,stroke_width=10).next_to(label1,LEFT)
        label2 = Text("Inbound").next_to(label1,DOWN)
        line2 = Line([0,0,0],[1,0,0],color=ORANGE,stroke_width=10).next_to(label2,LEFT)
        
        tracker = ValueTracker(0)
        tracker2 = ValueTracker(0)
        
        bee = ImageMobject("./images/bee_lowres.png").add_updater(update_bee).set_z_index(10)
        bee.height=0.2
        bee.original_points = bee.points.copy()
        nest = ImageMobject("./images/branch.png").rotate(PI / 4)
        nest.move_to([-0.7,0.1,0])
        nest.height = 2
        
        self.camera.frame.move_to([5,3,0]) 
        self.play(Write(title))
        self.add(tracker,tracker2)
        
        self.play(
            Write(label1),
            Write(label2),
            Create(line1),
            Create(line2),
        )
        self.pause()
        self.play(FadeIn(nest))
        self.add(bee)
        self.play(
            Create(outbound),
            tracker.animate.set_value(0.5),
            run_time=3
        )
        self.pause()
        self.play(
            Create(inbound),
            tracker.animate.set_value(1),
            run_time=2,
        )
        
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim Methods

class Methods(Slide):
    def construct(self):
        
        title = Title(r"Methods").set_z_index(100)
        
        blist = BulletedList("Framework for simulation", "Mathematical dye model", "Evaluation of performance")

        self.play(Write(title))
        self.play(Write(blist))
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim ClosestDist

with open('./ideas/positions2.json', 'r') as f:
    path2 = json.load(f)
path2 = np.array([p+[0.0] for p in path2])*0.05
f2= interp1d(np.linspace(0,1,1401), path2, axis=0)


class ClosestDist(Slide):
    def construct(self):
        
        title = Title(r"Error metrics \texttt{\#}1: Closest distance to home").set_z_index(100)
        
        outbound = ParametricFunction(f2, t_range=[0,1000/1400], color=BLUE)
        inbound = ParametricFunction(f2, t_range=[1000/1400,1], color=ORANGE)
        home = Dot(point=f2(0),color=GREEN).set_z_index(10)
        
        
        
        pathgroup = Group(home,outbound,inbound).move_to(DOWN+LEFT*2).rotate(PI/2)
        
        label1 = Text("Outbound").move_to(RIGHT*4+UP*2).set_z_index(100)
        line1 = Line([0,0,0],[1,0,0],color=BLUE,stroke_width=10).next_to(label1,LEFT).set_z_index(100)
        label2 = Text("Inbound").next_to(label1,DOWN).set_z_index(100)
        line2 = Line([0,0,0],[1,0,0],color=ORANGE,stroke_width=10).next_to(label2,LEFT).set_z_index(100)
        
        labelgroup = Group(label1,line1,label2,line2)
        
        
        
        self.play(Write(title),FadeIn(home))
        self.play(
            Write(label1),
            Write(label2),
            Create(line1),
            Create(line2),
        )
        self.pause()
        
        self.play(Create(outbound))
#         self.pause()
        self.play(Create(inbound))
        self.pause()
        self.play(
            pathgroup.animate.shift(RIGHT*9+UP*5).scale(6.5),
            labelgroup.animate.move_to(RIGHT*4+DOWN*3)
        )
        self.play(home.animate.scale(0.2),run_time=0.5)
        
        self.pause()
        
        errorline = DashedLine(start=home.get_center(),end=home.get_center()+UP*0.9)
        brace = Brace(errorline,direction=errorline.copy().rotate(-PI/2).get_unit_vector(),buff=0.2)
        b1text = brace.get_text("closest distance")
        
        self.play(Create(errorline),Create(brace),Write(b1text))
        
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim MemError

class MemError(Slide):
    def construct(self):
        
        title = Title(r"Error metrics \texttt{\#}2: Memory error")
        
        home_pos = RIGHT*3 + DOWN
        bee_pos = LEFT*3 + DOWN
        
        home = Dot(point=home_pos, color=GREEN)
        bee = ImageMobject("./images/bee_lowres.png").set_z_index(10).move_to(bee_pos).rotate(PI/3)
        bee.height=0.5
        
        
        homevector = Arrow(start=bee_pos,end=home_pos,buff=0,color=GREEN)
        hvtext = Text("actual home vector", font_size=30).move_to(homevector.get_center()+DOWN/2)
        
        angle = PI/2-3*PI/8
        x = np.linalg.norm(home_pos-bee_pos)*np.cos(angle)*np.cos(angle)
        y = np.linalg.norm(home_pos-bee_pos)*np.cos(angle)*np.sin(angle)
        mem_end = bee_pos + [x,y,0]
        memvector = Arrow(start=bee_pos,end=mem_end,buff=0,color=ORANGE)
        mvtext = Text("decoded memory", font_size=30).move_to(memvector.get_center()+LEFT*1.5+UP/2)
        
        errorline = DashedLine(end=homevector.get_end(),start=memvector.get_end())
        brace = Brace(errorline,direction=errorline.copy().rotate(PI/2).get_unit_vector(),buff=0.5)
        b1text = brace.get_text("memory error")
        
        self.play(Write(title))
        self.play(FadeIn(bee),FadeIn(home))
        self.play(GrowArrow(homevector))
        self.play(Write(hvtext),run_time=1)
        self.pause()
        self.play(GrowArrow(memvector))
        self.play(Write(mvtext),run_time=1)
        self.pause()
        self.play(Create(errorline))
        self.play(Create(brace),Write(b1text))
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim HeadError

class HeadError(Slide):
    def construct(self):
        
        title = Title(r"Error metrics \texttt{\#}3: Heading error")
        
        subtext = Text("*Only interesting during homing", font_size=20).move_to(DOWN*3.5+LEFT*4.5)
        
        home_pos = RIGHT*3 + DOWN
        bee_pos = LEFT*3 + DOWN
        
        home = Dot(point=home_pos, color=GREEN)
        bee = ImageMobject("./images/bee_lowres.png").set_z_index(10).move_to(bee_pos+UP*0.07).rotate(-3*PI/8)
        bee.height=0.35
        
        
        homevector = Arrow(start=bee_pos,end=home_pos,buff=0,color=GREEN)
        hvtext = Text("actual home vector", font_size=30).move_to(homevector.get_center()+DOWN/2)
        
        angle = PI/2-3*PI/8
        x = np.linalg.norm(home_pos-bee_pos)*np.cos(angle)*np.cos(angle)
        y = np.linalg.norm(home_pos-bee_pos)*np.cos(angle)*np.sin(angle)
        head_end = bee_pos + [x,y,0]
        headvector = Arrow(start=bee_pos,end=head_end,buff=0,color=BLUE)
        htext = Text("head direction", font_size=30).move_to(headvector.get_center()+LEFT*1.1+UP/2)
        
        errorline = DashedLine(end=homevector.get_end(),start=headvector.get_end())
        brace = Brace(errorline,direction=errorline.copy().rotate(PI/2).get_unit_vector(),buff=0.5)
        b1text = brace.get_text("heading error")
        
        self.play(Write(title))
        self.play(FadeIn(bee),FadeIn(home))
        self.play(GrowArrow(homevector))
        self.play(Write(hvtext),run_time=1)
        self.pause()
        self.play(GrowArrow(headvector))
        self.play(Write(htext),run_time=1)
        self.pause()
        self.play(Create(errorline))
        self.play(Create(brace),Write(b1text))
        self.play(Write(subtext),run_time=0.3)
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim Tortuosity

class Tortuosity(Slide):
    def construct(self):
        
        title = Title(r"Error metrics \texttt{\#}4: Tortuosity")
        
        
        home_pos = RIGHT*3 + DOWN
        bee_pos = LEFT*3 + DOWN
        
        home = Dot(point=home_pos, color=GREEN).set_z_index(10)
        bee = ImageMobject("./images/bee_lowres.png").set_z_index(10).move_to(bee_pos+UP*0.07).rotate(-3*PI/8)
        bee.height=0.35
        
        straight_line = Line(start=bee_pos,end=home_pos,buff=0,color=ORANGE,stroke_width=5)
#         text1 = Text("L", font_size=30).move_to(straight_line.get_center()+DOWN/2)
        
        brace1 = Brace(Line([-3,0,0]+DOWN,[3,0,0]+DOWN),direction=DOWN,buff=0.5)
        b1text = brace1.get_text("L")
        
        bent = ParametricFunction(
            lambda x: [x,-1*(x/3)**2,0],
            t_range=[-3,3],
            color="#3F00FF",
            stroke_width=5
        )
        
        bent2 = ParametricFunction(
            lambda x: [x,-1*(x/3)**2,0],
            t_range=[-3,2],
            color=ORANGE,
            stroke_width=5
        )
#         text2 = Text("C", font_size=30, color=WHITE).move_to(bent2.get_center()+UP)
        
        brace2 = Brace(Line([-3,0,0],[2,0,0]),direction=UP,buff=0.1)
        b2text = brace2.get_text("C")
        
        eq = MathTex(r'T=\frac{L}{C}').move_to(RIGHT*3+DOWN/2)
        
        self.play(Write(title))
        self.play(FadeIn(bee),FadeIn(home))
        self.play(Create(straight_line))
        self.pause()
        self.play(Create(bent))
        self.pause()
        self.play(Create(brace1),Write(b1text))
        self.pause()
        self.play(Create(bent2))
        self.play(Create(brace2),Write(b2text))
        self.pause()
        self.play(Group(bee,home,straight_line,brace1,b1text,brace2,b2text,bent,bent2).animate.shift(LEFT*3))
        self.play(Write(eq))
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim WeightPath

with open('./ideas/positions_weights.json', 'r') as f:
    path = json.load(f)
    path = np.array([[-p[0],p[1],0.0] for p in path])*0.018+0.1
    f = interp1d(np.linspace(0,1,3001), path, axis=0)

class WeightPath(Slide, MovingCameraScene):
    def construct(self):
        
        def update_bee(bee, dt):
            new_pos = f(tracker.get_value())
            tsub1 = f(np.maximum(0, tracker2.get_value()))
            
            diff = new_pos-tsub1
            angle = np.arctan2(diff[1], diff[0])
            bee.points = bee.original_points.copy()
                
            bee.rotate(angle-PI/2)
            bee.move_to(new_pos)
            
            tracker2.set_value(tracker.get_value())
            
        title = Title(r"Example path: Synaptic weights").set_z_index(100).shift(UP*3+RIGHT*5)
        
        outbound = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        inbound = ParametricFunction(f, t_range=[0.5,1], color=ORANGE)
        
        outbound2 = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        
        label1 = Text("Outbound").move_to([10,5,0])
        line1 = Line([0,0,0],[1,0,0],color=BLUE,stroke_width=10).next_to(label1,LEFT)
        label2 = Text("Inbound").next_to(label1,DOWN)
        line2 = Line([0,0,0],[1,0,0],color=ORANGE,stroke_width=10).next_to(label2,LEFT)
        
        tracker = ValueTracker(0)
        tracker2 = ValueTracker(0)
        
        bee = ImageMobject("./images/bee_lowres.png").add_updater(update_bee).set_z_index(10)
        bee.height=0.2
        bee.original_points = bee.points.copy()
        nest = ImageMobject("./images/branch.png").rotate(PI / 4)
        nest.move_to([-0.7,0.1,0])
        nest.height = 2
        
        self.camera.frame.move_to([5,3,0]) 
        self.play(Write(title))
        self.add(tracker,tracker2)
        
        self.play(
            Write(label1),
            Write(label2),
            Create(line1),
            Create(line2),
        )
        self.pause()
        self.play(FadeIn(nest))
        self.add(bee)
        self.play(
            Create(outbound),
            tracker.animate.set_value(0.5),
            run_time=3
        )
        self.pause()
        self.play(
            Create(inbound),
            tracker.animate.set_value(1),
            run_time=2,
        )
        
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim DyeMechanics

class DyeMechanics(Slide):
    def construct(self):
        
        title = Title(r"Dye mechanics").set_z_index(100)
        
        eq = MathTex(r"\frac{dc}{dt}", " = -", "k","c"," + ","u","\phi","(1-","T",")").shift(DOWN)
        
        transmittance = MathTex("T = ","10^{","A","}").next_to(eq,UP*2)
        absorbance = MathTex("A = ","\epsilon","\cdot","l","\cdot","(","c_{tot}","-c)").next_to(transmittance,UP)
        
        
        phi = MathTex("\phi"," [M/s] = \Phi [1] \cdot \Psi [M/J] \cdot W_{max} [J/s]").next_to(eq,DOWN*2)
        phi.set_color_by_tex("\phi",color=Colors.pure_red.value)
        
        
        self.play(Write(title))
        
        self.play(Write(eq),Write(absorbance),Write(transmittance))
        self.pause()
        
        self.play(absorbance.set_color_by_tex("\epsilon",color=Colors.pure_red.value).animate)
        self.pause()
        
        self.play(absorbance.set_color_by_tex("l",color=Colors.pure_red.value).animate,absorbance.set_color_by_tex("\epsilon",color=WHITE).animate)
        self.pause()
        
        self.play(absorbance.set_color_by_tex("l",color=WHITE).animate,absorbance.set_color_by_tex("c_{tot}",color=Colors.pure_red.value).animate)
        self.pause()
        
        self.play(absorbance.set_color_by_tex("c_{tot}",color=WHITE).animate)
        self.pause()
        
        self.play(eq.set_color_by_tex("k",color=Colors.pure_red.value).animate)
        self.pause()
        
        self.play(eq.set_color_by_tex("k",color=WHITE).animate, eq.set_color_by_tex("\phi",color=Colors.pure_red.value).animate)
        self.pause()
        
        self.play(Group(transmittance,absorbance,eq).animate.shift(UP))
        self.play(Write(phi))
        
#         self.play(eq.set_color_by_tex("\phi",color=WHITE).animate)
        
        self.pause()
        self.wait(0.1)

In [6]:
%%manim ParamSearch

class ParamSearch(Slide):
    def construct(self):
        
        img = ImageMobject("./images/phi-beta-epsilon.jpg").move_to([0,0,0])
        img.height = 8
#         box = BackgroundRectangle(img, color=WHITE)

        range1 = MathTex("10^{-4} < l[cm] < 10^{-3}  ")
        range2 = MathTex("10^4 < \epsilon_{max} [M^{-1}cm^{-1}] < 10^5").next_to(range1,DOWN)
        rGroup = Group(range1,range2).scale(0.7).shift(RIGHT*4.5+UP*2)
        
        t1 = Tex("$\epsilon \cdot l \\approx 15$").shift(RIGHT*4+UP)
        t2 = Tex("$\phi \\approx 0.0003$").shift(RIGHT*4)
        t3 = Tex("$\\beta \\approx 0.3$").shift(RIGHT*4+DOWN)
        
        tGroup = Group(t1,t2,t3).shift(DOWN*1.5+RIGHT*0.5)
        
        self.play(FadeIn(img))
        
        self.pause()
        self.play(img.animate.shift(LEFT*2.6))
        
        self.play(Write(range1),Write(range2))
        self.play(Write(t1),Write(t2),Write(t3))
        
        self.pause()
        self.wait(0.1)

[11/30/22 16:35:28] INFO     Animation 0 : Using cached data (hash :                           ]8;id=738002;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=968534;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py#75\75]8;;\
                             2724823860_1078173452_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=109652;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=897410;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py#75\75]8;;\
                             1164547788_3925602845_3362834651)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=700188;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=510049;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py#75\75]8;;\
                             1164547788_3822432335_2589047515)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=802705;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=590184;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py#75\75]8;;\
                             1164547788_1202656734_4057881607)                                                     

                    INFO     Animation 4 : Using cached data (hash :                           ]8;id=186576;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=400252;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/renderer/cairo_renderer.py#75\75]8;;\
                             1164547788_3409719442_3945587934)                                                     

                    INFO     Combining to Movie file.                                      ]8;id=296136;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=908012;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py#607\607]8;;\

                    INFO                                                                   ]8;id=279730;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=325545;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py#728\728]8;;\
                             File ready at                                                                         
                             '/home/jacob/path-integration-memory/notebooks/media/videos/n                         
                             otebooks/1080p60/ParamSearch.mp4'                                                     
                                                                                                                   

                    INFO     Rendered ParamSearch                                                      ]8;id=549332;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=995480;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene.py#240\240]8;;\
                             Played 5 animations                                                                   

                    INFO     Copied 5 animations to                                                    ]8;id=859298;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py\slide.py]8;;\:]8;id=10193;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py#174\174]8;;\
                             '/home/jacob/path-integration-memory/notebooks/slides/files/ParamSearch'              
                             and generated reversed animations                                                     

                    INFO     Slide 'ParamSearch' configuration written in                              ]8;id=868517;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py\slide.py]8;;\:]8;id=894338;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py#183\183]8;;\
                             '/home/jacob/path-integration-memory/notebooks/slides/ParamSearch.json'               

In [ ]:
%%manim Improvements

class Improvements(Slide):
    def construct(self):
        
        title = Title(r"Improvements").set_z_index(100)
        
        blist = BulletedList("Variable background activity", "Amplification layer")

        self.play(Write(title))
        self.play(Write(blist))
        
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim BasicPath

with open('./ideas/positions_basic.json', 'r') as f:
    path = json.load(f)
    path = np.array([[-p[0],p[1],0.0] for p in path])*0.018+0.1
    f = interp1d(np.linspace(0,1,3001), path, axis=0)

class BasicPath(Slide, MovingCameraScene):
    def construct(self):
        
        def update_bee(bee, dt):
            new_pos = f(tracker.get_value())
            tsub1 = f(np.maximum(0, tracker2.get_value()))
            
            diff = new_pos-tsub1
            angle = np.arctan2(diff[1], diff[0])
            bee.points = bee.original_points.copy()
                
            bee.rotate(angle-PI/2)
            bee.move_to(new_pos)
            
            tracker2.set_value(tracker.get_value())
            
        title = Title(r"Example path: Basic dye model").set_z_index(100).shift(UP*3+RIGHT*5)
        
        outbound = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        inbound = ParametricFunction(f, t_range=[0.5,1], color=ORANGE)
        
        outbound2 = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        
        label1 = Text("Outbound").move_to([10,5,0])
        line1 = Line([0,0,0],[1,0,0],color=BLUE,stroke_width=10).next_to(label1,LEFT)
        label2 = Text("Inbound").next_to(label1,DOWN)
        line2 = Line([0,0,0],[1,0,0],color=ORANGE,stroke_width=10).next_to(label2,LEFT)
        
        tracker = ValueTracker(0)
        tracker2 = ValueTracker(0)
        
        bee = ImageMobject("./images/bee_lowres.png").add_updater(update_bee).set_z_index(10)
        bee.height=0.2
        bee.original_points = bee.points.copy()
        nest = ImageMobject("./images/branch.png").rotate(PI / 4)
        nest.move_to([-0.7,0.1,0])
        nest.height = 2
        
    
        
        self.camera.frame.move_to([5,3,0]) 
        self.play(Write(title))
        self.add(tracker,tracker2)
        
        self.play(
            Write(label1),
            Write(label2),
            Create(line1),
            Create(line2),
        )
        self.pause()
        self.play(FadeIn(nest))
        self.add(bee)
        self.play(
            Create(outbound),
            tracker.animate.set_value(0.5),
            run_time=3
        )
        self.pause()
        self.play(
            Create(inbound),
            tracker.animate.set_value(1),
            run_time=2,
        )
        
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim AmpPath

with open('./ideas/positions_amp.json', 'r') as f:
    path = json.load(f)
    path = np.array([[-p[0],p[1],0.0] for p in path])*0.018+0.1
    f = interp1d(np.linspace(0,1,3001), path, axis=0)

class AmpPath(Slide, MovingCameraScene):
    def construct(self):
        
        def update_bee(bee, dt):
            new_pos = f(tracker.get_value())
            tsub1 = f(np.maximum(0, tracker2.get_value()))
            
            diff = new_pos-tsub1
            angle = np.arctan2(diff[1], diff[0])
            bee.points = bee.original_points.copy()
                
            bee.rotate(angle-PI/2)
            bee.move_to(new_pos)
            
            tracker2.set_value(tracker.get_value())
            
        title = Title(r"Example path: Amplification layer").set_z_index(100).shift(UP*3+RIGHT*5)
        
        outbound = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        inbound = ParametricFunction(f, t_range=[0.5,1], color=ORANGE)
        
        outbound2 = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        
        label1 = Text("Outbound").move_to([10,5,0])
        line1 = Line([0,0,0],[1,0,0],color=BLUE,stroke_width=10).next_to(label1,LEFT)
        label2 = Text("Inbound").next_to(label1,DOWN)
        line2 = Line([0,0,0],[1,0,0],color=ORANGE,stroke_width=10).next_to(label2,LEFT)
        
        tracker = ValueTracker(0)
        tracker2 = ValueTracker(0)
        
        bee = ImageMobject("./images/bee_lowres.png").add_updater(update_bee).set_z_index(10)
        bee.height=0.2
        bee.original_points = bee.points.copy()
        nest = ImageMobject("./images/branch.png").rotate(PI / 4)
        nest.move_to([-0.7,0.1,0])
        nest.height = 2
        
    
        
        self.camera.frame.move_to([5,3,0]) 
        self.play(Write(title))
        self.add(tracker,tracker2)
        
        self.play(
            Write(label1),
            Write(label2),
            Create(line1),
            Create(line2),
        )
        self.pause()
        self.play(FadeIn(nest))
        self.add(bee)
        self.play(
            Create(outbound),
            tracker.animate.set_value(0.5),
            run_time=3
        )
        self.pause()
        self.play(
            Create(inbound),
            tracker.animate.set_value(1),
            run_time=2,
        )
        
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim IntroduceModels

class IntroduceModels(Slide):
    def construct(self):
        
        def create_group(label,text,color=BLUE):
            l = Text(label, color=color)
            l.font_size = 50
            t = Text(text).next_to(l,DOWN)
            t.font_size = 30
            return Group(l,t)
        
        g1 = create_group("stone","Benchmark model from Stone")
        g2 = create_group("weights","Conceptual synaptic weight model",GREEN)
        g3 = create_group("dye basic","Basic version of our dye model",RED)
        g4 = create_group("dye var beta","Model with variable background activity",RED)
        g5 = create_group("dye amp","Model with amplification layer",RED)
        g6 = create_group("dye var beta + amp","Model with amplification layer",RED)
        
        self.play(FadeIn(g1))
        self.pause()
        self.play(AnimationGroup(g1.animate.move_to([-4,3,0]).scale(0.7)))
        self.play(FadeIn(g2))
        self.pause()
        self.play(AnimationGroup(g2.animate.move_to([4,3,0]).scale(0.7)))
        self.play(FadeIn(g3))
        self.pause()
        self.play(AnimationGroup(g3.animate.move_to([0,1,0]).scale(0.7)))
        self.play(FadeIn(g4))
        self.pause()
        self.play(AnimationGroup(g4.animate.move_to([-4,-1,0]).scale(0.7)))
        self.play(FadeIn(g5))
        self.pause()
        self.play(AnimationGroup(g5.animate.move_to([4,-1,0]).scale(0.7)))
        self.play(FadeIn(g6))
        self.pause()
        self.play(AnimationGroup(g6.animate.move_to([0,-3,0]).scale(0.7)))
        
        self.pause()
        self.wait(0.1)

In [2]:
%%manim ModelsResult

class ModelsResult(Slide):
    def construct(self): 
        img = ImageMobject("./images/models.jpg").move_to([0,0,0])
        img.height = 8
        
        self.play(FadeIn(img))
        
        self.pause()
        self.wait(0.1)

[11/30/22 16:33:15] INFO     Animation 0 : Partial movie file written in                   ]8;id=255107;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=516918;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             '/home/jacob/path-integration-memory/notebooks/media/videos/n                         
                             otebooks/1080p60/partial_movie_files/ModelsResult/450974505_6                         
                             22393074_223132457.mp4'                                                               

[11/30/22 16:33:16] INFO     Animation 1 : Partial movie file written in                   ]8;id=350764;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=785218;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             '/home/jacob/path-integration-memory/notebooks/media/videos/n                         
                             otebooks/1080p60/partial_movie_files/ModelsResult/34786186_34                         
                             09719442_160347518.mp4'                                                               

                    INFO     Combining to Movie file.                                      ]8;id=52666;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=514764;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py#607\607]8;;\

                    INFO                                                                   ]8;id=834166;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=105165;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene_file_writer.py#728\728]8;;\
                             File ready at                                                                         
                             '/home/jacob/path-integration-memory/notebooks/media/videos/n                         
                             otebooks/1080p60/ModelsResult.mp4'                                                    
                                                                                                                   

                    INFO     Rendered ModelsResult                                                     ]8;id=956700;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=50279;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim/scene/scene.py#240\240]8;;\
                             Played 2 animations                                                                   

                    INFO     Copied 2 animations to                                                    ]8;id=686586;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py\slide.py]8;;\:]8;id=895105;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py#174\174]8;;\
                             '/home/jacob/path-integration-memory/notebooks/slides/files/ModelsResult'             
                             and generated reversed animations                                                     

                    INFO     Slide 'ModelsResult' configuration written in                             ]8;id=306467;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py\slide.py]8;;\:]8;id=478063;file:///home/jacob/path-integration-memory/venv/lib/python3.8/site-packages/manim_slides/slide.py#183\183]8;;\
                             '/home/jacob/path-integration-memory/notebooks/slides/ModelsResult.json'              

In [ ]:
%%manim EvalResult

paths = ["./images/weights.png","./images/dye.png","./images/dye-beta.png","./images/dye-cheat.png","./images/dye-beta-cheat.png",]
titles = ["Weights","Dye basic","Dye var beta","Dye amp","Dye var beta + amp"]

class EvalResult(Slide):
    def construct(self):
        
        def create_img_and_title(path,title,side=1):
            img = ImageMobject(path).move_to([side*3,10,0])
            img.height=7
            title_obj = Text(title).next_to(img,UP)
            title_obj.font_size = 30
#             title_obj.height = 0.3
            return Group(img,title_obj)
        
        StoneGroup = create_img_and_title("./images/stone.png","Stone",-1)
        StoneGroup.move_to([-3,-0.08,0])
        self.add(StoneGroup)
        
        for path,title in zip(paths,titles):
            group = create_img_and_title(path,title)
            self.play(group.animate.move_to([3,0,0]))
            self.pause()
            self.play(group.animate.move_to([3,-20,0]))
            
        self.pause()
        self.wait(0.1)

In [ ]:
%%manim Conclusions

class Conclusions(Slide):
    def construct(self): 
        title = Title(r"Conclusions")
        
        bullets = BulletedList(
            "Can such dyes be used to implement the Stone model?",
            "What changes to the network are required?",
            "Can our results be related to biology?",
        ).scale(0.8).shift(LEFT)
        
        yes = Text("Yes", color=GREEN,font_size=35).next_to(bullets,RIGHT)
        minimal = Text("Minimal", color=GREEN,font_size=35).next_to(bullets,RIGHT)
        kinda = Text("Somewhat", color=GREEN,font_size=35).next_to(bullets,RIGHT)
        
        self.play(Write(title))
        self.play(Write(bullets))
        
        self.pause()
        self.play(bullets.animate.fade_all_but(0))
        self.pause()
        self.play(Write(yes))
        self.pause()
        
        self.play(FadeOut(yes),bullets.animate.fade_all_but(1))
        self.pause()
        self.play(Write(minimal))
        self.pause()
        
        self.play(FadeOut(minimal),bullets.animate.fade_all_but(2))
        self.pause()
        self.play(Write(kinda))
        self.pause()
        self.play(FadeOut(kinda))
        self.pause()
        
        
        self.play(FadeOut(bullets),FadeOut(title))
        self.play(Write(Text("Thanks for listening!",font_size=60)))

        self.pause()
        self.wait(0.1)